# Courses Revision and Cheat Sheets

## School of Mathematics

### SCQF Level 10

- [MATH10003 Financial Mathematics (2016)](FiM/FiM.pdf)
- [MATH10067 Honours Complex Variables (2017)](HCoV/HCoV.pdf)
- [MATH10004 Likelihood (2017)](Lik/Lik.pdf)
- MATH10005 Linear Statistical Modelling (2016) Part 1 [2](LSM/Part2.pdf)
- [MATH10007 Stochastic Modelling (2017)](SMo/SMo.pdf)
- [MATH10028 Theory of Statistical Inference (2018)](TSI/TSI.html)

### SCQF Level 11

- [MATH11186 Nonparametric Regression Models (2018)](NPR/NPR.html)
- [MATH11024 The Analysis of Survival Data (2017)](ASD/ASD.pdf)

## School of Informatics
- [INFR11130 Machine Learning and Pattern Recognition (2018)](MLPR/index.html)